In [1]:
import pandas as pd
import os
import arcpy
from utils import *

# Get Data

In [2]:
# get parcels from the database
# network path to connection files
filePath = "F:/GIS/PARCELUPDATE/Workspace/"
# database file path 
sdeBase    = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")
sdeTabular = os.path.join(filePath, "Tabular.sde")

arcpy.env.workspace = 'memory'
# # clear memory workspace
# arcpy.management.Delete('memory')

# overwrite true
arcpy.env.overwriteOutput = True
# Set spatial reference to NAD 1983 UTM Zone 10N
sr = arcpy.SpatialReference(26910)

We need to get data for parcels, join census assign to geographies

In [ ]:
census_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/28'
df_census = get_fs_data(census_url)
df_census_2022 = df_census.loc[(df_census['year_sample'] == 2022) & (df_census['sample_level'] == 'block group')]

In [ ]:
# parcel development layer polygons
parcel_db = sdeCollect + "\\SDE.Parcel\\SDE.Parcel_History_Attributed"
# query 2022 rows
sdf_units = pd.DataFrame.spatial.from_featureclass(parcel_db)
sdf_units = sdf_units.loc[sdf_units['YEAR'] == 2022]
sdf_units.spatial.sr = sr

In [ ]:
census_variable_list = pd.read_csv('Lookup_Lists/census_variables.csv')
df_census_2022_include = df_census_2022.loc[df_census_2022['variable_code'].isin(census_variable_list['variable_code'])]
block_group_pivot = df_census_2022_include.pivot(index='block_group', columns='variable_code', values='value')


In [ ]:
units_attributed = pd.merge(sdf_units, block_group_pivot, left_on='TRPAID', right_on='TRPAID', how='left')